In [4]:
import subprocess

import networkx as nx
from networkx import MultiDiGraph

import matplotlib.pyplot as plt

import pandas as pd

In [7]:
def find_import(test_file):
    try:
        return subprocess.check_output(["grep", "Require", test_file]).decode('utf-8').replace("Require Import ","").split(".\n")[:-1] #Select everything but the last '\n' that was split
    except subprocess.CalledProcessError as e:
        print('No dependencies')
        return []

In [8]:
find_file_locations = lambda : subprocess.check_output(['bash', '-c', "find . -type f -name '*.v'"]).decode('utf-8').split('\n')
remove_file_extension = lambda x : x[:-2]
remove_file_name = lambda x : "/".join(x.split("/")[:-1])
remove_file_path = lambda x : x.split("/")[-1]
path_to_logic = lambda x : x[2:].replace('/','.')
path_to_R = lambda x : x[2:] if '/' not in x[2:] else '"'+x[2:]+'"'

In [10]:
df = pd.DataFrame({
    'file_location' : find_file_locations()[:-1]
})
df['file'] = df['file_location'].apply(remove_file_path)
df['file_name'] = df['file'].apply(remove_file_extension)
df['file_path'] = df['file_location'].apply(remove_file_name)
df['logical_path'] = df['file_path'].apply(path_to_logic)
df['R_path'] = df['file_path'].apply(path_to_R)
df['dependencies']=df['file_location'].apply(find_import)
df['import_name'] = df['logical_path']+ '.' +df['file_name']

No dependencies


In [11]:
list_of = list()
for i in df['dependencies']:
    for j in i:
        list_of.append(j)
counter_of = lambda y : list_of.count(y)
df['in_deg'] = df['import_name'].apply(counter_of)


In [12]:
df.sort_values(['in_deg', 'out_deg'], inplace=True, ascending=(True, False))

KeyError: 'out_deg'